# Part 03
----
아파트 매매와 인구이동 데이터의 비교분석을 용이하게 하기 위한 전처리 과정을 행한다.

우선적으로 년월의 구분 방법이 다른 것을 같은 형태로 만든다. 또한 데이터 조작의 편의를 위해서 칼럼 이름을 영문으로 전부 고치고 비교 가능한 칼럼들은 같은 이름으로 통일한다.

이후 대부분의 코드는 행정코드의 불일치를 고치는 과정이다. 행정코드의 불일치는 크게 두가지 이유로 남아있다.

하나는 아파트 매매의 경우 주소를 무조건적으로 처음 두단어로 축약하여 코드로 매핑을 하였지만 인구 이동의 경우 행정코드 앞 다섯자리만 남기는 방식으로 전처리를 거쳤기 때문에 방식의 차이로 인해 코드 불일치가 일어났다. 일반적으로 인구 이동 데이터에서의 행정코드가 더 세밀한 단위까지 구분되어 있다. 이를 해결하기 위해 인구 이동 데이터에서 행정코드 -> 주소 -> 축약주소 -> 행정코드 과정을 거쳐 최대한 아파트 매매의 행정코드와 일치시킨다.

나머지 하나는 행정구역의 변화로 인하여 두개의 데이터 프레임은 같은 지역임에도 서로 다른 행정코드를 가지고 있는 경우도 있으며 다행히 이 경우는 약 10가지 경우 남짓하여 손으로 직접 수정을 가한다.

In [1]:
%matplotlib inline
import re

In [2]:
# setting the folder path and read in data
path = 'raw_data\\'
df_pop = pd.read_csv(path + '02_pop.csv', encoding='utf-8')
df_estate = pd.read_csv(path + '01_estate.csv', encoding='utf-8')

In [3]:
# quick initial check
df_pop.head()

,year,month,area_code,move_in,foreign,reason,house_total,hh_head,head_age,head_male,mov_multi,mov_ppl,mov_male
0,2006,1,11110,1,0,1,309,258,42.344961,178,121,588,326
1,2006,1,11110,1,0,2,32,8,48.500000,6,8,50,23
2,2006,1,11110,1,0,3,134,129,41.945736,87,59,264,135
3,2006,1,11110,1,0,4,12,11,38.000000,3,8,26,8
4,2006,1,11110,1,0,5,2,2,30.500000,1,0,2,1


In [4]:
df_estate.head()

,area,price,built,year,month,area_code
0,43.01,4520,2000,2006,1,36110
1,43.01,4495,2000,2006,1,36110
2,43.01,4495,2000,2006,1,36110
3,43.01,4520,2000,2006,1,36110
4,43.01,4520,2000,2006,1,36110


In [5]:
estate_column_order = [
    'year', 'month', 'area_code', 'area', 'price', 'built'
]

In [6]:
df_estate = df_estate[estate_column_order]

In [7]:
df_estate.head()

,year,month,area_code,area,price,built
0,2006,1,36110,43.01,4520,2000
1,2006,1,36110,43.01,4495,2000
2,2006,1,36110,43.01,4495,2000
3,2006,1,36110,43.01,4520,2000
4,2006,1,36110,43.01,4520,2000


In [8]:
# we see that the number of area codes do not match up between the dataframes
df_pop.groupby('area_code').count().shape

(264, 12)

In [9]:
df_estate.groupby('area_code').count().shape

(228, 5)

In [10]:
# we try to see the difference in the area codes used in both dataframes
estate_area_code = df_estate.area_code[~df_estate.area_code.duplicated()]
estate_area_code_missing = estate_area_code[~estate_area_code.isin(df_pop.area_code)]
estate_area_code_missing.values

array([41280, 41190, 41130, 41110, 41270, 41170, 41460, 47110, 45110, 43110], dtype=int64)

In [11]:
pop_area_code = df_pop.area_code[~df_pop.area_code.duplicated()]
pop_area_code_missing = pop_area_code[~pop_area_code.isin(df_estate.area_code)]
pop_area_code_missing.values

array([28720, 41111, 41113, 41115, 41117, 41131, 41133, 41135, 41171,
       41173, 41195, 41197, 41199, 41271, 41273, 41281, 41285, 41287,
       41461, 41463, 41465, 41730, 43111, 43113, 43710, 44730, 44830,
       45111, 45113, 47111, 47113, 48160, 48190, 49110, 49130, 49710,
       49720, 44131, 44133, 48121, 48123, 48125, 48127, 48129, 43112, 43114], dtype=int64)

In [12]:
# just a custom function to use as an aggregate method
def agg_nunique(series):
    return series.nunique()

In [13]:
address_to_code = pd.read_csv(path + '01_address_to_code.csv', index_col=0).area_code
code_to_address = pd.read_csv(path + '01_code_to_address.csv', index_col=0).address

In [14]:
# function that returns the first word or two words of the string
subaddress_regex = re.compile(u'(\S+\s|\S+)\s*(\S*)')
def shrink_address(address):
    return ''.join(subaddress_regex.search(address).groups())

In [15]:
pd.DataFrame(data={
    'area_code': estate_area_code_missing.values,
    'address': estate_area_code_missing.map(code_to_address).apply(shrink_address)
})

,address,area_code
695,경기도 고양시,41280
2930,경기도 부천시,41190
3214,경기도 성남시,41130
4091,경기도 수원시,41110
5208,경기도 안산시,41270
5635,경기도 안양시,41170
7097,경기도 용인시,41460
11666,경상북도 포항시,47110
23775,전라북도 전주시,45110
25205,충청북도 청주시,43110


In [16]:
# comparing with the table above, we see that in many cases, the problem arises from
# the area codes being too granular in the population dataframe
pd.DataFrame(data={
    'area_code': pop_area_code_missing.values,
    'address': pop_area_code_missing.map(code_to_address).apply(shrink_address)
})

,address,area_code
800,인천광역시 옹진군,28720
1019,경기도 수원시,41111
1033,경기도 수원시,41113
1047,경기도 수원시,41115
1061,경기도 수원시,41117
1074,경기도 성남시,41131
1088,경기도 성남시,41133
1102,경기도 성남시,41135
1130,경기도 안양시,41171
1144,경기도 안양시,41173


In [17]:
# run the address code through the process explained above
temp_address = df_pop.area_code.map(code_to_address)
temp_address = temp_address.apply(shrink_address)
temp_address = temp_address.map(address_to_code)
df_pop.area_code = temp_address

In [18]:
# we repeat the process as before trying to see the changes
estate_area_code = df_estate.area_code[~df_estate.area_code.duplicated()]
estate_area_code_missing = estate_area_code[~estate_area_code.isin(df_pop.area_code)]
estate_area_code_missing.values

array([], dtype=int64)

In [19]:
pd.DataFrame(data={
    'area_code': estate_area_code_missing.values,
    'address': estate_area_code_missing.map(code_to_address).apply(shrink_address)
})

,address,area_code


In [20]:
pop_area_code = df_pop.area_code[~df_pop.area_code.duplicated()]
pop_area_code_missing = pop_area_code[~pop_area_code.isin(df_estate.area_code)]
pop_area_code_missing.values

array([28720, 41730, 43710, 44730, 44830, 48150, 48190, 49110, 49130,
       49710, 49720], dtype=int64)

In [21]:
pd.DataFrame(data={
    'area_code': pop_area_code_missing.values,
    'address': pop_area_code_missing.map(code_to_address).apply(shrink_address)
})

,address,area_code
800,인천광역시 옹진군,28720
1561,경기도 여주군,41730
1888,충청북도 청원군,43710
2114,충청남도 연기군,44730
2198,충청남도 당진군,44830
3000,경상남도 마산시,48150
3028,경상남도 진해시,48190
3252,제주도 제주시,49110
3266,제주도 서귀포시,49130
3280,제주도 북제주군,49710


In [22]:
# as there are only 11 values left to fix, we tidy up things manually
# 옹진군 is actually a collection of small islands, hence it is reasonable there are no appartments in the area
# however we make it part of 중구 for ease of use in manipulating data with consistency
# 여주군 has changed to 여주시
# 청원군 has become part of 청주시
# 연기군 has become part of 세종특별자치시
# 당진군 has changed to 당진시
# 마산시, 진해시 have become part of 창원시
# 제주도 has changed to 제주특별자치도

In [23]:
# creating resources to fix area codes
def area_code_fix(x):
    if x in area_code_fix_map.keys():
        return area_code_fix_map[x]
    else:
        return x

In [24]:
area_code_fix_map = {
    28720: 28110, # 용진군: 중구
    41730: 41670, # 여주군: 여주시
    43710: 43110, # 청원군: 청주시
    44730: 36110, # 연기군: 세종특별자치시
    44830: 44270, # 당진군: 당진시
    48150: 48110, # 마산시: 창원시
    48190: 48110, # 진해시: 창원시
    49110: 50110, # 제주시: 제주시
    49130: 50130, # 서귀포시: 서귀포시
    49710: 50110, # 북제주군: 제주시
    49720: 50130, # 남제주군: 서귀포시
}

In [25]:
df_pop.area_code = df_pop.area_code.apply(area_code_fix)

In [26]:
# one last sanity check
estate_area_code = df_estate.area_code[~df_estate.area_code.duplicated()]
estate_area_code_missing = estate_area_code[~estate_area_code.isin(df_pop.area_code)]
estate_area_code_missing.values

array([], dtype=int64)

In [27]:
pop_area_code = df_pop.area_code[~df_pop.area_code.duplicated()]
pop_area_code_missing = pop_area_code[~pop_area_code.isin(df_estate.area_code)]
pop_area_code_missing.values

array([], dtype=int64)

In [28]:
# exporting scrubbed dataframes
df_estate.to_csv(path + '03_estate.csv', index=False, encoding='utf-8')
df_pop.to_csv(path + '03_pop.csv', index=False, encoding='utf-8')